In [1]:
#| default_exp inference

In [2]:
#| export

from cam import CAM, GradCAM, GradCAMpp, SmoothGradCAMpp, ScoreCAM
from utils.visualize import visualize, reverse_normalize
from utils.imagenet_labels import label2idx, idx2label
from matplotlib import pyplot as plt
import numpy as np
import preprocess
import os

import torch

# def generate_saliency(img_path, model, out_dir, inf_img_name, sal_method):
def generate_saliency(img_arr, model, sal_method):
#name of sal_method must be same as the function
    
#     img_tensor = preprocess.img_to_tensor(img_path)
    ngpu = torch.cuda.current_device()
    for gpu_id in range(ngpu):
        gpu = torch.cuda.get_device_name(gpu_id)
        print(f"GPU:{gpu_id} {gpu}")
    
    img_tensor = preprocess.img_to_tensor(img_arr)
    
    gpu_reference_tensor = next(model.parameters())    
    img_tensor = img_tensor.type_as(gpu_reference_tensor)
    target_layer = model.layer4[1].conv2
    wrapped_model = eval(sal_method)(model, target_layer)
    cam, idx = wrapped_model(img_tensor)
    
    
    if sal_method in ['GradCAM', 'GradCAMpp', 'SmoothGradCAMpp']:
        cam = cam.cpu()
    
#     fig, ax = plt.subplots()
    print(idx)
    img = reverse_normalize(img_tensor)
    heatmap = visualize(img, cam)
    heatmap = np.transpose(heatmap.squeeze(), (1, 2, 0))
    
#     ax.imshow(heatmap, cmap='turbo', alpha = 0.8)
    
    classes = open("/project/validating_attribution_techniques/shardul/classes.txt").read().splitlines()
#   ax.set_title(f"Class: {classes[idx]}")
#     plt.axis('off')
    
#     if not os.path.exists(out_dir):
#         os.makedirs(out_dir, exist_ok=True)
#     fig.savefig(f"{out_dir}/{inf_img_name+'-'+idx2label[idx].replace(' ','-')}.jpeg")
#     np.save(f"{out_dir}/{inf_img_name+'-'+idx2label[idx].replace(' ','-')}.npy", heatmap)
        
    return cam, heatmap

In [3]:
#| hide
from nbdev.export import nb_export
nb_export('inference.ipynb', './')